# Chapter 5-2. 파이썬 일급함수 - 클로저
## Closure : 핵심은 불변 상태를 기억한다는 것!

- Definition: A Closure is a function object that remembers values in enclosing scopes even if they are not present in memory.
- 파이썬 변수 범위(Scope)
- Global 선언
- 클로저 사용 이유 : 서버 프로그래밍은 동시성(Concurrency) 제어가 핵심임!
  - => 즉 한정된 메모리 공간에 여러 자원이 접근하면 교착상태(Dead Lock, Shutdown, Lace conditino)에 빠지기 마련임
  - => 이를 해결하기 위해, 메모리를 공유하지 않고 메세지 전달로 처리하기 위한,
  - => 클로저는 기억을 공유는 하되 **변경되지 않는! (Immutable, Read Only)** 구조를 적극적으로 사용한다. => 함수형 프로그래밍과 연결됨
  - 클로저는 불변자료구조 및 atom, STM를 통해 -> 멀티스레드(Coroutine) 프로그래밍에 강점을 가진다. 
- Class -> Closure 구현

In [1]:
def func_v1(a):
    print(a)
    print(b)

func_v1(10)

10


NameError: name 'b' is not defined

In [3]:
b = 20
def func_v2(a):
    print(a)
    print(b)

func_v2(10)

10
20


In [7]:
c = 30
def func_v3(a):
    print(a)
    print(c)
    c = 40 # 안에 같은 이름이 있을 땐 local variable이 우선임
    
func_v3(10)

10


UnboundLocalError: local variable 'c' referenced before assignment

In [8]:
c = 30
def func_v3(a):
    c = 40
    print(a)
    print(c)
     # 안에 같은 이름이 있을 땐 local variable이 우선임
     # 
print('>>', c)    
func_v3(10)

>> 30
10
40


In [10]:
c = 30
def func_v3(a):
    global c
    print(a)
    print(c)
    c = 40
     # 안에 같은 이름이 있을 땐 local variable이 우선임
     # 
print('>>', c)    
func_v3(10)
print('>>>', c)

>> 30
10
30
>>> 40


# Chapter 5-2. 파이썬 일급함수
## Closure : 핵심은 불변 상태를 기억한다는 것!
- Definition: A Closure is a function object that remembers values in enclosing scopes even if they are not present in memory.
  - 외부에서 호출된 함수의 변수값, 상태(레퍼런스) 복사 후 저장 -> 후에 접근(엑세스) 가능
- 클로저 사용 이유 : 서버 프로그래밍은 동시성(Concurrency) 제어가 핵심임!
  - => 즉 한정된 메모리 공간에 여러 자원이 접근하면 교착상태(Dead Lock, Shutdown, Lace conditino)에 빠지기 마련임
  - => 이를 해결하기 위해, 메모리를 공유하지 않고 메세지 전달로 처리하기 위한,
  - => 클로저는 기억을 공유는 하되 **변경되지 않는! (Immutable, Read Only)** 구조를 적극적으로 사용한다. => 함수형 프로그래밍과 연결됨
  - 클로저는 불변자료구조 및 atom, STM를 통해 -> 멀티스레드(Coroutine) 프로그래밍에 강점을 가진다. 
- Class -> Closure 구현
- 

In [12]:
a = 100
print(a + 100)
print(a + 1000)

200
1100


In [14]:
# 결과 누적(함수 사용)
print(sum(range(1, 51)))
print(sum(range(51, 101)))

1275
3775


In [1]:
# 클래스 이용
class Averager():
    def __init__(self):
        self._series = []
        
    def __call__(self, v): # callable이 있으면 클래스를 함수처럼 사용할 수 있음
        self._series.append(v)
        print('inner >> {} / {}'.format(self._series, len(self._series)))
        return sum(self._series) / len(self._series)

In [2]:
# instance generation
averager_cls = Averager()

print(dir(averager_cls)) # __call__ 이 있음 => 함수로써 호출 가능
print(averager_cls(10)) # 클래스 인스턴스 생성한 걸 마치 함수처럼 호출하고 있음
print(averager_cls(30))
print(averager_cls(50)) # 계속 누적되고 있음 : 상태를 기억한다는 것임. 

['__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_series']
inner >> [10] / 1
10.0
inner >> [10, 30] / 2
20.0
inner >> [10, 30, 50] / 3
30.0


In [4]:
averager_cls.__dict__()

TypeError: 'dict' object is not callable

## Closure 심화
- 외부에서 호출된 함수의 변수값, 상태(레퍼런스) 복사 후(=>snapshot) 저장 -> 후에 접근(엑세스) 가능

In [23]:
# Closure 사용
def closure_ex1(): # 내부에 함수가 또 있는 구조
    # Free variable: 내가 사용하려는 함수 바깥에서 선언된 변수
    # Closure scope
    series = []
    def averager(v):
        series.append(v)
        print('inner >>> {} / {}'.format(series, len(series)))
        return sum(series) / len(series)
    
    return averager

In [25]:
avg_closure1 = closure_ex1()
print(avg_closure1(10))
print(avg_closure1(30))
print(avg_closure1(50))

inner >>> [10] / 1
10.0
inner >>> [10, 30] / 2
20.0
inner >>> [10, 30, 50] / 3
30.0


In [33]:
avg_closure2 = closure_ex1()
print(avg_closure2(50))

inner >>> [50] / 1
50.0


In [32]:
# function inspection
print(dir(avg_closure1))
print()
print(dir(avg_closure1.__code__)) # co들이 많음 # co_freevars
print(avg_closure1.__code__.co_freevars) # 자유변수 출력하니 클로즈 영역의 시리즈를 가지고 있네
print()
print(dir(avg_closure1.__closure__))
print()
print(avg_closure1.__closure__[0].cell_contents)
print()

['__annotations__', '__call__', '__class__', '__closure__', '__code__', '__defaults__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__get__', '__getattribute__', '__globals__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__kwdefaults__', '__le__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']

['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'co_argcount', 'co_cellvars', 'co_code', 'co_consts', 'co_filename', 'co_firstlineno', 'co_flags', 'co_freevars', 'co_kwonlyargcount', 'co_lnotab', 'co_name', 'co_names', 'co_nlocals', 'co_posonlyargcount', 'co_stacksiz

In [45]:
# 잘못된 클로저 사용의 예
def closure_ex2():
    # Free variable
    cnt = 0
    total = 0
    
    def averager(v):
        nonlocal cnt, total
        cnt += 1 # len
        total += v # sum
        return total / cnt 
    return averager

In [47]:
avg_closure2 = closure_ex2()
print(avg_closure2(10))
print(avg_closure2(35))
print(avg_closure2(40))


10.0
22.5
28.333333333333332


## 5-4. 파이썬 일급함수 - 데코레이터(Decorator)
#### 장점
1. 중복 제거, 코드 간결, 공통 함수 작성
2. 로깅, 프레임워크, 유효성 체크 -> 공통 기능
3. 조합해서 사용 용이
   
#### 단점
1. 가독성 감소?
2. 특정 기능에 한정된 함수는 -> 단일 함수로 작성하는 것이 유리할 수도
3. 디버깅 불편

In [3]:
import time

def perf_clock(func):
    # free variables 영역
    def perf_clocked(*args):
        # 함수 시작 시간
        st = time.perf_counter()
        # 함수 실행
        result = func(*args)
        # 함수 소요 시간 (끝시간 - 시작시간)
        et = time.perf_counter() - st
        # 실행 함수명
        name = func.__name__
        # 함수 매개변수
        arg_str = ', '.join(repr(arg) for arg in args) # 제네레이터
        # 결과 출력
        print('[%0.5fs] %s(%s) -> %r' % (et, name, arg_str, result))
        return result
    return perf_clocked

In [4]:
def time_func(seconds):
    time.sleep(seconds)
    
def sum_func(*numbers):
    return sum(numbers)

In [6]:
# 데코레이터 미사용
none_deco1 = perf_clock(time_func)
none_deco2 = perf_clock(sum_func)

In [10]:
print(none_deco1, none_deco1.__code__.co_freevars) # 함수를 인자로 넘길 수 있기 때문에 프리변수에 함수
print(none_deco2, none_deco2.__code__.co_freevars)

print('=' * 40, 'Called None Decorator -> time_func')
print()
none_deco1(1.5)
print('-' * 40, 'Called None Decorator -> sum_func')
print()
none_deco2(100, 200, 300, 400, 500)

<function perf_clock.<locals>.perf_clocked at 0x106232dc0> ('func',)
<function perf_clock.<locals>.perf_clocked at 0x106232550> ('func',)
======================================== Called None Decorator -> time_func

[1.50504s] time_func(1.5) -> None
---------------------------------------- Called None Decorator -> sum_func

[0.00000s] sum_func(100, 200, 300, 400, 500) -> 1500


1500

In [14]:
@perf_clock
def time_func(seconds):
    time.sleep(seconds)
@perf_clock
def sum_func(*numbers):
    return sum(numbers)

print('=' * 40, 'Called Decorator -> time_func')
print()
time_func(1.5)
print('-' * 40, 'Called Decorator -> sum_func')
print()
sum_func(100, 200, 300, 400, 500)

======================================== Called Decorator -> time_func

[1.50146s] time_func(1.5) -> None
---------------------------------------- Called Decorator -> sum_func

[0.00000s] sum_func(100, 200, 300, 400, 500) -> 1500


1500

In [ ]:
import time

def perf_clock(func):
    # free variables 영역
    def perf_clocked(*args):
        # 함수 시작 시간
        st = time.perf_counter()
        # 함수 실행
        result = func(*args)
        # 함수 소요 시간 (끝시간 - 시작시간)
        et = time.perf_counter() - st
        # 실행 함수명
        name = func.__name__
        # 함수 매개변수
        arg_str = ', '.join(repr(arg) for arg in args) # 제네레이터
        # 결과 출력
        print('[%0.5fs] %s(%s) -> %r' % (et, name, arg_str, result))
        return result
    return perf_clocked

In [15]:
time.time()

1666865264.3384871

In [16]:
time.perf_counter()

14712.465633875

In [17]:
import time
import functools

def clock(func):
    @functools.wraps(func)
    def perf_clocked(*args, **kwargs):
        st = time.time()
        result = func(*args, **kwargs)
        et = time.time() - st
        name = func.__name__
        arg_lst = []
        if args:
            arg_lst.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = [f'{key}={value}' for key, value in sorted(kwargs.items())]
            arg_lst.append(', '.join(pairs))
        arg_str = ', '.join(arg_lst)
        print('[%0.5fs] %s(%s) => %r' % (et, name, arg_str, result))
        return result
    return perf_clocked